In [1]:
#!pip install --user torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0

In [2]:
#!pip install timm

### Imports

In [1]:
from fastai.vision.all import *
import tqdm

from timm import create_model
from fastai.vision.learner import _update_first_layer
import gc

In [3]:
stage = 'train'
path = Path('src')
img_path = path / stage

mask_path = path / 'masks'

df = pd.read_csv(path / f'{stage}.csv')

In [4]:
enc_path = 'models/resnet34_encoder.pkl'
class_learn_path = 'models/class_resnet34_learner.pth'
mask_learn_path =  'models/mask_resnet34_learner.pth'

In [5]:
df.tail(1)

,id,annotation,width,height,cell_type,plate_time,sample_date,sample_id,elapsed_timedelta
73584,ffdb3cc02eef,249775 2 250477 6 251180 8 251882 11 252585 12 253288 14 253992 14 254695 16 255398 17 256102 17 256805 17 257509 17 258212 17 258917 16 259621 15 260326 13 261031 11 261736 9 262442 6,704,520,cort,11h59m00s,2020-11-01,cort[debris]_D9-3_Vessel-384_Ph_4,0 days 11:59:00


### Label functions for classification & segmentation

In [6]:
def class_label_func(fn):
    #return re.search('(.*)_Phase.*',fn.stem).group(1)
    return df.loc[df.id == fn.stem, 'cell_type'].unique()[0]

In [10]:
def mask_label_func(fn):
    return mask_path / fn.name

### Use the same train/val split troughout the switch training, to get reliable metrics.

In [7]:
df['is_valid'] = False

val_ids = np.random.choice(df.id.unique(), size = int(606 * 0.2))
df.loc[df.id.isin(val_ids), 'is_valid'] = True

In [8]:
#is_val_fns = df.loc[df.is_valid == 1, 'file_name'].unique()
#is_train_fns = df.loc[df.is_valid == 0, 'file_name'].unique()

def is_valid(fn):
    return fn.stem in val_ids

In [9]:
def MySplitter():
    def _inner(o):
        train_idx = o.argwhere(is_valid, negate = True)
        val_idx = o.argwhere(is_valid)    
        return train_idx, val_idx
    return _inner

### Training Parameters

In [12]:
TASKS = ['class', 'mask']
SIZES = [(224, 303), (336, 455), (448, 606), None] #None equals native image size of (520, 704)
DEBUG = True
BS_MULTS = [1, 1, 1, 1]#[40, 18, 10, 6]

### Shrink dataset for debuging / prototyping

In [13]:
items = get_image_files(img_path)
print(len(items))
unique_files = df.file_name.unique()
if DEBUG:
    unique_files = np.random.choice(unique_files, 1000)
items = L([fn for fn in items if fn.stem in unique_files])
print(len(items))
fn = items[0]
img_rgb = load_image(fn)

### Build Dataloaders

In [15]:
def get_all(path):
    items = get_image_files(path)
    if DEBUG:
        items = np.random.choice(items, 500)
    return L(*items)

In [61]:
def get_dls(src_path, task, size, bs_mult = 1):
    if task == 'mask':
        print('Build dataloaders for segmentation.')
        TaskBlock = (MaskBlock(codes = np.array(['bg', 'cell'])),)
        label_func = mask_label_func
        bs = 1 * bs_mult
        get_items = get_all #get_annotated
        
    if task == 'class':
        print('Build dataloaders for classification.')
        TaskBlock = (CategoryBlock(),)
        label_func = class_label_func
        bs = int(8 * bs_mult)
        get_items = get_all
        
    db = DataBlock(
        blocks = (ImageBlock(), *TaskBlock),
        get_items = get_items,
        get_y = label_func,
        splitter = MySplitter(),
        #item_tfms = Resize((448, 606)),
        batch_tfms = [*aug_transforms(size = size, 
                                    flip_vert = True, 
                                    max_rotate = 180., 
                                    max_warp = 0.
                                   ),
                      Normalize.from_stats(*imagenet_stats)
                     ],
        n_inp = 1
    )
    print(f'Using batch size of: {bs} and image size of: {size}')
    return db.dataloaders(src_path, bs = bs)

### Build Model

In [62]:
def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): 
        return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): 
        return cut(model)
    else: 
        raise NamedError("cut must be either integer or function")

### Workaround to change the classification learner head from 9 classes (pretraining) to 3. 

In [63]:
### Create same model as used in pretraining
#body = create_timm_body('resnet34', pretrained = False)
#n_out = 9
#nf = num_features_model(nn.Sequential(*body.children()))
#head = create_head(nf, n_out, ps = 0.5)
#model = nn.Sequential(body, head)

### Create new learner from that model
#dls = get_dls(img_path, 'class', (224, 303), bs_mult = 1)
#learn = Learner(
#    dls,
#    model,
#    metrics = accuracy,
#    splitter = default_split,
#).to_fp16()

### Load weights from pretraining
#learn.unfreeze()
#learn.load('class_resnet34_learner')

### Substitute last layer by one with needed output features
#learn.model[-1][-1] = nn.Linear(in_features = 512, out_features = 3, bias = False)
#apply_init(model[-1][-1], nn.init.kaiming_normal_)

### Save model weights
#learn.save('class_resnet34_learner_sartorius', with_opt = False)

### Build Learner

In [64]:
def get_learner(task, dls, arch, learner_ex, encoder_ex = False, with_opt = False):

    body = create_timm_body(arch, pretrained = True)
    n_out = get_c(dls)

    if task == 'mask':
        img_size = dls.one_batch()[0].shape[-2:]
        model = models.unet.DynamicUnet(body, n_out, img_size, self_attention = True)
        metrics = [Dice, JaccardCoeff]

    if task == 'class':
        nf = num_features_model(nn.Sequential(*body.children()))
        head = create_head(nf, n_out, ps = 0.5)
        model = nn.Sequential(body, head)
        apply_init(model[1], nn.init.kaiming_normal_)
        metrics = accuracy

    print(f'Create learner for task {task}')
    learn = Learner(
        dls,
        model,
        metrics = metrics,
        splitter = default_split
    ).to_fp16()
    
    if learner_ex[TASKS.index(task)]:
        print(f'Load pretrained model')
        #learn = load_learner(f'{task}_{arch}_learner.pkl', cpu = False)
        #learn.dls = dls
        learn.load(f'{task}_{arch}_learner', with_opt = with_opt)
    
    if encoder_ex:
        print('Load encoder from previous task')
        load_model(f'{arch}_encoder.pkl', learn.model[0], opt=None, with_opt=False)
    else:
        print('Start training from scratch.')

    return learn

## Switch Training

In [41]:
def switch_training(tasks, sizes, epochs, rounds, learner_ex, encoder_ex, lr = 1e-3, arch = 'resnet34', with_opt = False): #efficientnet_b3a
    print('Multitask training with: ', arch)
    for k, size in enumerate(sizes):
        for i in range(rounds):
            print(f'Start round no {i+1} with image size {size}')
            for task in tasks:
                if task == 'class':
                    eps = epochs[0]
                else:
                    eps = epochs[1]
                dls = get_dls(img_path, task, size, bs_mult = 1)
                learn = get_learner(task, dls, arch, learner_ex, encoder_ex, with_opt = with_opt)
                learn.fine_tune(eps, lr, cbs = CutMix(), freeze_epochs = 1)

                # Save encoder for all tasks
                encoder = get_model(learn.model)[0]
                save_model(f'{arch}_encoder.pkl', encoder, opt = None, with_opt=False)
                encoder_ex = True
                # Save learner for specific task
                #learn.export(f'{task}_{arch}_learner.pkl')
                learn.save(f'{task}_{arch}_learner')
                with_opt = True
                learner_ex[TASKS.index(task)] = True

                #Cleanup
                if (size != None) & (task == 'mask'):
                    print('Clean up')
                    del(dls)
                    del(learn)
                    torch.cuda.empty_cache()
                    gc.collect()
                else:
                    learn.export('mask_learner_final.pkl')
                print(f'Done with task: {task}')
            print(f'Finish round no: {i+1} with image size: {size}')

In [44]:
switch_training(TASKS, SIZES, epochs = [3, 7], rounds = 1, learner_ex = [True, True], encoder_ex = True)

Multitask training with:  resnet34
Start round no 1 with image size (224, 303)
Build dataloaders for classification.
Using batch size of: 8 and image size of: (224, 303)
Create learner for task class
Load pretrained model
Load encoder from previous task


epoch,train_loss,valid_loss,accuracy,time
0,1.306740,0.358232,0.827160,00:03


epoch,train_loss,valid_loss,accuracy,time
0,0.985211,0.319932,0.876543,00:04
1,0.894811,0.237170,0.925926,00:04
2,0.954591,0.281618,0.839506,00:04


Done with task: class
Build dataloaders for segmentation.
Using batch size of: 1 and image size of: (224, 303)
Create learner for task mask
Load pretrained model
Load encoder from previous task


epoch,train_loss,valid_loss,dice,jaccard_coeff,time
0,0.181521,0.238498,0.627143,0.456816,00:29


...

epoch,train_loss,valid_loss,accuracy,time
0,0.669046,0.024379,1.000000,00:10
1,0.648728,0.063650,0.975904,00:10
2,0.607382,0.067390,0.975904,00:10


Done with task: class
Build dataloaders for segmentation.
Using batch size of: 1 and image size of: None
Create learner for task mask
Load pretrained model
Load encoder from previous task


epoch,train_loss,valid_loss,dice,jaccard_coeff,time
0,0.121166,0.125349,0.745365,0.594089,01:20


epoch,train_loss,valid_loss,dice,jaccard_coeff,time
0,0.101037,0.129411,0.768670,0.624260,01:23
1,0.101380,0.120360,0.763430,0.617377,01:22
2,0.127924,0.125561,0.771399,0.627867,01:22
3,0.103065,0.124629,0.738253,0.585104,01:22
4,0.100548,0.146188,0.676349,0.510972,01:22
5,0.097417,0.136004,0.762022,0.615538,01:22
6,0.111194,0.132303,0.760009,0.612915,01:22


Done with task: mask
Finish round no: 1 with image size: None


## Export learners

In [30]:
task = 'class'
arch = 'resnet34'

dls = get_dls(img_path, task, (224, 303))
learn = get_learner(task, dls, arch, [True, True], False)

Build dataloaders for classification.
Using batch size of: 8 and image size of: (224, 303)
Create learner for task class
Load pretrained model
Start training from scratch.


In [31]:
learn.export('class_final.pkl')

In [32]:
task = 'mask'
arch = 'resnet34'

dls = get_dls(img_path, task, (224, 303))
learn = get_learner(task, dls, arch, [True, True], False)

Build dataloaders for segmentation.
Using batch size of: 1 and image size of: (224, 303)
Create learner for task mask
Load pretrained model
Start training from scratch.


In [33]:
learn.export('mask_final.pkl')